In [ ]:
#!pip install openpyxl

# Electoral College

After numerous failed attempts to parse the official archives.gov website there were numerous issues causing road blocks.  The main issue was the rowspan / colspan in the html table.

So I manually copied/pasted the data from archives.gov into a publicly available [Google Sheet](https://docs.google.com/spreadsheets/d/1RPrW58dFiQprCtTSYP76wjHGlq0p_az3uJypPI0-PoY/edit?usp=sharing) with the 3rd row as a machine-friendly header row mapping the appropriate candidate column to `ev_dem` and `ev_rep` column header.

The code below iterates over each "year" worksheet and combines into a single CSV.  It also fleshes out `state_po` (2 letter abbreviation) and `state_fips` (2 digit zero paded federal identifier)

In [1]:
import re
import pandas as pd

# ---------------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------------
SPREADSHEET_ID = "1RPrW58dFiQprCtTSYP76wjHGlq0p_az3uJypPI0-PoY"
xlsx_url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=xlsx"

expected_header = ["state", "state_ev_total", "ev_dem", "ev_rep"]

# ---------------------------------------------------------------------
# OFFICIAL STATE → POSTAL + FIPS MAPPING (includes DC)
# ---------------------------------------------------------------------
state_lookup = pd.DataFrame({
    "state": [
        "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut",
        "Delaware","District of Columbia","Florida","Georgia","Hawaii","Idaho","Illinois",
        "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts",
        "Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada",
        "New Hampshire","New Jersey","New Mexico","New York","North Carolina","North Dakota",
        "Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island","South Carolina",
        "South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington",
        "West Virginia","Wisconsin","Wyoming"
    ],
    "state_po": [
        "AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA",
        "KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM",
        "NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA",
        "WV","WI","WY"
    ],
    "state_fips": [
        "01","02","04","05","06","08","09","10","11","12","13","15","16","17","18","19",
        "20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35",
        "36","37","38","39","40","41","42","44","45","46","47","48","49","50","51","53",
        "54","55","56"
    ]
})
print("Starting... This can take 5 to 15 seconds or more")
# ---------------------------------------------------------------------
# READ + PARSE GOOGLE SHEET
# ---------------------------------------------------------------------
sheets = pd.read_excel(xlsx_url, sheet_name=None, header=None, dtype=object)

dfs = []
headers_seen = []

for sheet_name, raw in sheets.items():
    # Skip non-year or "info"
    if sheet_name.lower() == "info" or not re.fullmatch(r"\d{4}", sheet_name):
        continue

    raw = raw.iloc[:, :4]
    if raw.shape[0] < 3:
        print(f"⚠️  Skipping {sheet_name}: fewer than 3 rows")
        continue

    # Use 3rd row (index 2) as header
    header = (
        raw.iloc[2, :4]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.lower()
        .tolist()
    )
    headers_seen.append(tuple(header))

    if header != expected_header:
        print(f"⚠️  {sheet_name} header differs.\n"
              f"    Found:  {header}\n"
              f"    Expect: {expected_header}")

    # Skip top 3 rows
    df = raw.iloc[3:, :4].copy()
    df.columns = header
    df = df.dropna(how="all")

    # Trim whitespace
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()

    # Merge state_po + fips
    df = df.merge(state_lookup, how="left", on="state")

    # Reorder columns: year first
    df.insert(0, "year", sheet_name)
    df = df[["year", "state", "state_po", "state_fips", "state_ev_total", "ev_dem", "ev_rep"]]

    dfs.append(df)

# ---------------------------------------------------------------------
# CONSISTENCY + SAVE
# ---------------------------------------------------------------------
if headers_seen:
    counts = pd.Series(headers_seen).value_counts()
    print("\n=== Header Consistency Check (first 4 cols) ===")
    for hdr, n in counts.items():
        print(f"{list(hdr)}  → {n} sheet(s)")

df_ev = pd.concat(dfs, ignore_index=True)

# Optional: coerce numeric columns
num_cols = ["state_ev_total", "ev_dem", "ev_rep"]
for c in num_cols:
    df_ev[c] = pd.to_numeric(df_ev[c], errors="coerce").astype("Int64")
    #df_ev[c] = pd.to_integer(df_ev[c], errors="coerce")

df_ev.to_csv("data/electoral_college_votes_by_state_year.csv", index=False)

print(f"\n✅ Combined {len(dfs)} sheets → {len(df_ev)} total rows")
print("✅ Saved to data/electoral_college_votes_by_state_year.csv")


Starting... This can take 5 to 15 seconds or more

=== Header Consistency Check (first 4 cols) ===
['state', 'state_ev_total', 'ev_dem', 'ev_rep']  → 15 sheet(s)

✅ Combined 15 sheets → 780 total rows
✅ Saved to data/electoral_college_votes_by_state_year.csv
